In [2]:
import requests
import json
import pandas as pd
import numpy as np
import time
from os.path import exists
from datetime import datetime

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.0.0)/charset_normalizer (2.0.9) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [5]:
# Libraries for GC API
import os
from dotenv import load_dotenv

In [14]:
import mitosheet

In [6]:
# requires a valid key to use the GC API
# see docs https://api.canada.ca/en/homepage 
# create a .env file in this repository and add GC_API_KEY="YOUR_GC_API_KEY"
# make sure to add .env filetype to gitignore
load_dotenv()
gc_key = os.environ.get("GC_API_KEY")

# temporarily suppresses SettingWithCopyWarning
pd.options.mode.chained_assignment = None 

In [16]:
input_data = "../5a-Matching/output/matched.csv"
df = pd.read_csv(input_data, low_memory=False, dtype="str")
df_input = df

In [8]:
len(df)

803491

In [11]:
len(df[df.latitude.isna() & df.x.isna()])
# len(df[df.latitude.isna()])

163590

In [25]:
sample_size = 100
if (len(df_input) > sample_size):
    df = df_input.sample(sample_size)
else:
    sample_size = len(df)

In [28]:
# we need to geocode if a results has na for latitude AND has ratio < 80 OR ratio na

# ok how to get started with geocoding. 
# let's just test the bing script



In [26]:
mitosheet.sheet(df, analysis_to_replay="id-rmmopdyskz")

MitoWidget(analysis_data_json='{"analysisName": "id-rmmopdyskz", "analysisToReplay": null, "code": [], "stepSu…

In [ ]:
from mitosheet import *; register_analysis("id-rmmopdyskz");
    
# Reordered column keep_match
df_columns = [col for col in df.columns if col != 'keep_match']
df_columns.insert(51, 'keep_match')
df = df[df_columns]


In [ ]:
from mitosheet import *; register_analysis("id-heawxepiyk");
    
# Reordered column keep_match
df_columns = [col for col in df.columns if col != 'keep_match']
df_columns.insert(51, 'keep_match')
df = df[df_columns]


In [8]:
# define parameters for osm api call
url_gc = 'https://national-address-register-statcan.api.canada.ca:443/v1/addresses/search'

JSONS = []
JSONS_CITIES = []

In [15]:
# clean dataset for queries
df.fillna('', inplace=True)

# define api query
df['gc_request_street'] = df['street_no'] + ', ' + df['street_name'] + ', ' + df['city'] + ', ' + df['province']
reqs_gc = list(df['gc_request_street'])

In [ ]:
# Filter to only do geocoding on specific results

# no lat lon
# has well parsed address
# hasn't already been geocoded - use idx

In [ ]:
df['gc_request_street']

In [ ]:
# define query
def gc_query(query, query_type):
    # set interval for requests
    time.sleep(3) 
    coords = requests.get(url, params=params, headers=headers)
    print("osm query " + query_type + ": " + query)
    coords_gc = requests.get(url_gc, params=params_gc, headers=headers_gc)
    return coords.json()

In [29]:
for i in range(len(reqs_gc)):
    query_gc = reqs_gc[i]
    
    
    time.sleep(1) 
        
    # we first try facility name street address,
    # if that fails, we try street address and then city 
    # which one we use is recorded in the geo_source column
        
    print(str(i + 1) + ': ' + query_gc)
    

     # try GC API
    params_gc = {'qstr': query_gc}
    headers_gc = {'user_key': gc_key}
    coords_gc = requests.get(url_gc, params=params_gc, headers=headers_gc)
    
    print(coords_gc)
#                 print("trying gc api with query: " + query_gc)    
    if (coords_gc.status_code == 200):
        resp = coords_gc.json()
        print('[3] gc street address found')
#         df['geo_source'].iloc[i] = "gc_street_address"
        print(resp)
    else:
        print('no gc street address found')

    print("\n")
#     JSONS.append(resp)

#     if resp!=[]:
#         print("\n")
# #         print(resp)

1: 2963, ELLINGTON AVE, Prince George, BC
<Response [200]>
[3] gc street address found
{'meta': {'prepared': '2022-11-03T20:53:40.7290826+00:00', 'test': False, 'contentLanguage': ['en', 'fr']}, 'search': {'searchStyle': 'simple', 'criteria': 'Query: 2963, ELLINGTON AVE, Prince George, BC  ', 'total': 1}, 'data': [{'addressId': '0d1eddc8-1fcc-4e36-bbb8-2308349d906f', 'civicNumber': {'prefix': None, 'number': '2963', 'suffix': {'code': None}}, 'streetName': 'Ellington', 'streetType': {'code': 'AVE'}, 'streetDirection': {'code': None}, 'cityName': {'en': 'Prince George', 'fr': 'Prince George'}, 'province': {'code': '59'}, 'country': {'code': 'CA', 'name': {'en': 'Canada', 'fr': 'Canada'}}, 'mailingAddress': {'streetName': 'ELLINGTON', 'streetType': {'code': 'AVE'}, 'streetDirection': {'code': None}, 'municipalityName': 'PRINCE GEORGE', 'province': {'code': 'BC'}, 'country': {'code': 'CA', 'name': {'en': 'Canada', 'fr': 'Canada'}}, 'postalCode': 'V2N0G4'}, 'location': {'locationID': 'dcee